If this notebook is being run on Google Collab, install the requisite packages before anything else.

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install plasmapy==0.7.0

# Charged Particle Radiography Workbook

[charged_particle_radiography]: https://docs.plasmapy.org/en/latest/ad/diagnostics/charged_particle_radiography.html

This workbook creates an example electric or magnetic field, then uses the PlasmaPy [charged_particle_radiography] module to create a synthetic proton radiograph through those fields.

In [ ]:
import astropy.constants as const
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from plasmapy.diagnostics import charged_particle_radiography as cpr
from plasmapy.plasma.grids import CartesianGrid

## Creating Example Fields

[CartesianGrid]: https://docs.plasmapy.org/en/stable/api/plasmapy.plasma.grids.CartesianGrid.html#plasmapy.plasma.grids.CartesianGrid

[example notebook]:https://docs.plasmapy.org/en/stable/notebooks/plasma/grids_cartesian.html

In order to input the example fields, we will create a PlasmaPy [CartesianGrid] object (as described in this [example notebook]) and fill it with the analytical fields from one of two models:

1. The electric field produced by a sphere of Gaussian potential. 
2. A cylindrical region of straight magnetic field

You can change between these examples by changing the variable "example" below from "e-sphere" to "b-cylinder".

In [ ]:
example = 'e-sphere'
#example = 'b-cylinder'

# For both cases, create a CartesianGrid from -1 to 1 mm in each dimension and 100 grid points.
L = 1 * u.mm
grid = CartesianGrid(-L, L, num=100)

if example == 'e-sphere':
    # Try increasing phi0 to increase the electric field magnitude
    phi0 = 5e4 * u.V
    #phi0 = 2e5 * u.V
    
    # Create a spherical potential with a Gaussian radial distribution
    radius = np.linalg.norm(grid.grid, axis=3)
    arg = (radius / (L / 3)).to(u.dimensionless_unscaled)
    potential = phi0 * np.exp(-(arg ** 2))

    # Calculate E from the potential
    Ex, Ey, Ez = np.gradient(potential, grid.dax0, grid.dax1, grid.dax2)
    Ex = -np.where(radius < L / 2, Ex, 0)
    Ey = -np.where(radius < L / 2, Ey, 0)
    Ez = -np.where(radius < L / 2, Ez, 0)
    
    Bx = np.zeros(grid.shape)*u.T
    By = np.zeros(grid.shape)*u.T
    Bz = np.zeros(grid.shape)*u.T
    
elif example == 'b-cylinder':
    # Try increasing B0 to increase the magnetic field strength.
    B0 = 10 * u.T
    #B0 = 50 * u.T
    
    radius = np.sqrt(grid.pts0**2 + grid.pts1**2)
    Ex = np.zeros(grid.shape)*u.V/u.m
    Ey = np.zeros(grid.shape)*u.V/u.m
    Ez = np.zeros(grid.shape)*u.V/u.m
    
    Bx = np.zeros(grid.shape)*u.T
    By = np.zeros(grid.shape)*u.T
    Bz = np.where(radius < 0.3*u.mm, B0, 0*u.T)

# Add those quantities to the grid
grid.add_quantities(E_x=Ex, E_y=Ey, E_z=Ez,
                    B_x=Bx, B_y=By, B_z=Bz)

# Print a summary of the grid, showing the fields we have added
print(grid)

Lets plot the fields, just to make sure they look like we expect

In [ ]:
fig, axarr = plt.subplots(ncols=2, figsize=(8, 6))
fig.subplots_adjust(wspace=0.35)

# Define a slice to plot only a subset of the points
s = slice(None,None,3)

for ax in axarr:
    ax.set_aspect('equal')
    ax.set_xlabel("X (mm)")
    ax.set_ylabel("Z (mm)")
    
ax = axarr[0]
ax.set_title("E (at y=0)")
ax.quiver(grid.pts0[s, 50, s].to(u.mm).value, 
          grid.pts2[s, 50, s].to(u.mm).value, 
          grid['E_x'][s,50,s].value, grid['E_z'][s,50,s].value,
          angles='xy', scale=3e6)

ax = axarr[1]
ax.set_title("B (at y=0)")
ax.quiver(grid.pts0[s, 50, s].to(u.mm).value, 
          grid.pts2[s, 50, s].to(u.mm).value, 
          grid['B_x'][s,50,s].value, grid['B_z'][s,50,s].value,
          angles='xy', scale=200);

## Runing the Particle Tracing Algorithm

[astropy.units.Quantity]: https://docs.astropy.org/en/stable/units/quantity.html#quantity
[example tutorial]: https://docs.plasmapy.org/en/latest/notebooks/diagnostics/charged_particle_radiography_particle_tracing.html
[Tracker]:https://docs.plasmapy.org/en/latest/api/plasmapy.diagnostics.charged_particle_radiography.Tracker.html#plasmapy.diagnostics.charged_particle_radiography.Tracker

This notebook is adapted from the [example tutorial] in the PlasmaPy documentation.

First we will define the location of the source and the center of the detector plane, then create the [Tracker] object which will run the simulation.

**Things to Try**
- Try setting the source and detector positions in spherical or cylindrical coordinates by using the appropriate units (as in the commented-out lines below).
- Try moving the source and detector positions to vary the system magnification.
- For the magnetic field cylinder example (which is not spherically symmetric), how does the radiograph change if the source-detector vector is at an angle to the B-field?

In [ ]:
source = (0 * u.mm, -10 * u.mm, 0 * u.mm)
detector = (0 * u.mm, 100 * u.mm, 0 * u.mm)
#source = (10 * u.mm, 0 * u.deg, 180 * u.deg)
#detector = (100 * u.mm, 0 * u.deg, 0 * u.deg)

sim = cpr.Tracker(grid, source, detector, verbose=True)

[create_particles()]: https://docs.plasmapy.org/en/latest/api/plasmapy.diagnostics.charged_particle_radiography.Tracker.html#plasmapy.diagnostics.charged_particle_radiography.Tracker.create_particles

Next we will create the test particles with [create_particles()]

**Things to Try**
- Including more or fewer particles will result in less or more noise (but also affects the computation time)
- Increase the particle energy, and notice how the resulting deflections become smaller.

In [ ]:
sim.create_particles(5e4, 3 * u.MeV, max_theta=np.pi / 15 * u.rad, particle="p")

[add_wire_mesh()]: https://docs.plasmapy.org/en/stable/api/plasmapy.diagnostics.proton_radiography.SyntheticProtonRadiograph.html#plasmapy.diagnostics.proton_radiography.SyntheticProtonRadiograph.add_wire_mesh
       
In many proton radiography experiments, a wire fiducial mesh is placed between the source and the object plasma to provide a known spatial reference. To add a mesh fiducial to this simulation, uncomment the bottom line below. 

**To Try**
- Vary the position of the mesh relative to the source, which changes the magnification of the mesh on the detector. 

In [ ]:
location = np.array([0, -2, 0]) * u.mm
extent = (1.5 * u.mm, 1.5 * u.mm)
nwires = (9, 12)
wire_diameter = 25 * u.um
# Uncomment this line to add a wire fiducial mesh
#sim.add_wire_mesh(location, extent, nwires, wire_diameter)

[run()]: https://docs.plasmapy.org/en/latest/api/plasmapy.diagnostics.charged_particle_radiography.Tracker.html#plasmapy.diagnostics.charged_particle_radiography.Tracker.run

Now we will run the simulation by calling [run()].

**Things to Try**
- Setting the field_weighting keyword to 'nearest neighbor' will make the simulation run faster, but the interpolated fields will be less accurate.

In [ ]:
sim.run(field_weighting='volume averaged')

[synthetic_radiograph()]: https://docs.plasmapy.org/en/latest/api/plasmapy.diagnostics.charged_particle_radiography.synthetic_radiograph.html#plasmapy.diagnostics.charged_particle_radiography.synthetic_radiograph

Finally, we will create a synthetic radiograph histogram by using the [synthetic_radiograph()] function.

**To Try**
- Increasing the size of the radiograph will change the field of view.
- Increasing or decreasing the number of bins will change the resolution, but also affects the amount of noise per pixel. 

In [ ]:
size = np.array([[-1, 1], [-1, 1]]) * 1.5 * u.cm
bins = [200, 200]
hax, vax, intensity = cpr.synthetic_radiograph(sim, size=size, bins=bins)

# Make the plot
fig, ax = plt.subplots(figsize=(8, 8))
plot = ax.pcolormesh(
    hax.to(u.cm).value,
    vax.to(u.cm).value,
    intensity.T,
    cmap="Blues_r",
    shading="auto",
)
cb = fig.colorbar(plot)
cb.ax.set_ylabel("Intensity")
ax.set_aspect("equal")
ax.set_xlabel("X (cm), Image plane")
ax.set_ylabel("Z (cm), Image plane")
ax.set_title("Synthetic Proton Radiograph");

Now you can try re-running this script with different input keywords to see how they change the radiograph! It is best to do this by making changes, then re-running the entire notebook with "Cell > Run All".